In [1]:
%pip install plotly

DEPRECATION: Loading egg at c:\users\dell\anaconda3\lib\site-packages\huggingface_hub-0.28.1-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
import torch
from transformers import (
    BertForQuestionAnswering,
    BertTokenizerFast,
)
from scipy.special import softmax
import plotly.express as px
import pandas as pd
import numpy as np

In [3]:
model_name="deepset/bert-base-cased-squad2"

In [4]:
tokenizer=BertTokenizerFast.from_pretrained(model_name)
model=BertForQuestionAnswering.from_pretrained(model_name)

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:

context = """Subhash Chandra Bose passed through quarters inhabited by Englishmen and also met a
 large number of them in the tram cars. The British using these cars were purposely rude and offensive to Indians in various ways. The sensitive mind of Subhash revolted against such insulting and rude behavior of the British.
 On many occasions, there was an exchange of hot words between him and misbehaving British.
   Majority of the students of the Presidency College, where he studied, were free thinkers. 
   The college continued to be a storm centre and was looked upon by the British Government "as a hotbed of sedition, rendezvous of revolutionaries" and was frequently searched by the police.
 The first two years of his life were greatly influenced by the group, which styled itself as the neo-Vivekananda group and Subhash developed intellectually during this period. The group generally
   followed the teachings of Rama Krishna and Vivekananda with special emphasis on social service as
 means of spiritual development and was non-aligned to a revolutionary group. 
 The shock of the Great World War roused his political consciousness. He graduated at the age of 22 and enrolled himself for the postgraduate with experimental psychology as a special subject. 
His father, however, wanted him to go to England to appear for the Indian Civil Services. 
In spite of his mental reservations, Subhash took it as a challenge. In England, 
he was greatly impressed with the freedom allowed to students at Cambridge. Every student behaved in a dignified manner. Notwithstanding his preoccupation with his studies,
 he displayed his public spirit and fearlessness throughout his stay in England. He and K. L. Gouba were selected by the Indian Majlis, to represent the British Government the difficulties the Indian students encountered for admission to the University Officers' Training Corps. Though he took a harsh view of the British high handedness and racial arrogance, 
 he did admire their qualities which exacted him. He himself behaved there in a dignified way and was of the view that Indians who go abroad, must consider themselves to be unofficial ambassadors of the country, who should uphold their country's prestige. 
 He was quite serious in purpose and disliked anybody wasting time on trivialities."""
question = "Who were selected by the Indian Majlis?"


In [15]:
inputs=tokenizer(question,context,return_tensors='pt')

In [16]:
inputs

{'input_ids': tensor([[  101,  2627,  1127,  2700,  1118,  1103,  1890, 18285,  6137,   136,
           102, 12859, 16481,  1324, 17595, 24880,  2085,  1194,  7541,  9375,
          1118,  1483,  2354,  1105,  1145,  1899,   170,  1415,  1295,  1104,
          1172,  1107,  1103, 14172,  3079,   119,  1109,  1418,  1606,  1292,
          3079,  1127,  3007,  1193, 14708,  1105,  5810,  1106,  5888,  1107,
          1672,  3242,   119,  1109,  7246,  1713,  1104, 12859, 16481,  1324,
         11733,  1174,  1222,  1216, 27296,  1105, 14708,  4658,  1104,  1103,
          1418,   119,  1212,  1242,  6070,   117,  1175,  1108,  1126,  3670,
          1104,  2633,  1734,  1206,  1140,  1105,  1940,  1116,  3962, 22300,
          1418,   119, 23670,  1104,  1103,  1651,  1104,  1103, 18037,  1531,
           117,  1187,  1119,  2376,   117,  1127,  1714,  1341,  1468,   119,
          1109,  2134,  1598,  1106,  1129,   170,  4162,  2642,  1105,  1108,
          1350,  1852,  1118,  1103,  

In [17]:
with torch.no_grad():
    output=model(**inputs)

    

In [18]:
start_score,end_score=softmax(output.start_logits)[0],softmax(output.end_logits)[0]

In [19]:
start_idx = np.argmax(start_score)
end_idx = np.argmax(end_score)
confidence_score = (start_score[start_idx] + end_score[end_idx]) /2
answer_ids = inputs.input_ids[0][start_idx: end_idx + 1]
answer_tokens = tokenizer.convert_ids_to_tokens(answer_ids)
answer = tokenizer.convert_tokens_to_string(answer_tokens)

In [26]:
if answer != tokenizer.cls_token:
     print(answer, confidence_score)
else:
     print(None, confidence_score)


K. L. Gouba 0.9873538017272949


In [21]:
question

'Who were selected by the Indian Majlis?'